# End-to-End Regression Project using Indian Rental Data

## 📘 Regression: Problem Overview

### What is Regression?

**Regression** is a machine learning task where we predict a **continuous numeric value** from input features.

Common regression problems include:

* 🏠 House price / rental prediction
* 💼 Salary prediction
* 📈 Sales or demand estimation
* ⚡ Energy or cost estimation

All of these involve predicting a **numeric value**, not a category.

### Type of Learning: Supervised

This is a **supervised learning** problem.

* We are given **input features (X)**
* We are given the **correct output (y)**
* The model learns the relationship between them from **labeled training data**

In this notebook:

* **Inputs:** house attributes (area, city, bedrooms, furnishing, etc.)
* **Target:** monthly rent

### Dataset Used

* Based on a **MagicBricks rental listings dataset (India)** [Kaggle Dataset Link](https://www.kaggle.com/datasets/pranayjagtap06/indian-rental-housing-price-dataset)
* Slightly **modified for teaching purposes**

  * to demonstrate preprocessing
  * to handle missing values
  * to build clean end-to-end pipelines

### What We’ll Do in This Notebook

We will walk through a **complete regression workflow**:

1. Understand and explore the data
2. Prepare features and handle missing values
3. Split data into train / validation / test
4. Build and compare regression models
5. Evaluate on unseen test data

The goal is to understand **how regression is applied end to end in practice**, not just to train a model.


## 0) Setup

* Detailed setup instructions are available in the **README** of the repository
* **Download the dataset** and place it inside the `data/` folder
* This project uses **uv** for environment and dependency management
* Clone the repo and follow the README — all required libraries (pandas, scikit-learn, matplotlib, etc.) will be installed automatically
Once setup is complete, you’re ready to run this notebook end to end.


## 1) Load the dataset

We start by loading the CSV into a DataFrame.
Then we preview a few rows to understand the columns.

In [ ]:
# Pandas is the standard library for working with tables in Python.
import pandas as pd

DATA_PATH = r"data\cities_magicbricks_rental_prices.csv"
df = pd.read_csv(DATA_PATH)

df.head(5)






### Dataset Overview & Feature Description

Each row in the dataset represents a **single rental property listing**.
Below is a brief description of the available features:

| Feature        | Description                                           |
| -------------- | ----------------------------------------------------- |
| **house_type** | Title or type of the property                         |
| **locality**   | Locality or neighborhood of the property              |
| **city**       | City where the property is located                    |
| **area**       | Property area in square feet (sq ft)                  |
| **beds**       | Number of bedrooms                                    |
| **bathrooms**  | Number of bathrooms                                   |
| **balconies**  | Number of balconies                                   |
| **furnishing** | Furnishing status of the property                     |
| **area_rate**  | Area rate in Indian Rupees (₹) per sq ft              |
| **rent**       | Monthly rent in Indian Rupees (₹) *(target variable)* |


> **Notes**
> - `rent` is the target we want to predict. It is continuous → **regression**.
> - We have numeric features (`area`, `beds`, …) and categorical features (`city`, `furnishing`, …).
> - `house_type` is free text. We will skip it in the first baseline model.

## 2) Quick data checks

We check size, data types, and missing values.
This tells us what preprocessing we need.

In [ ]:
# Quick dataset size summary (readable format)
from IPython.display import display, Markdown

rows, cols = df.shape
display(Markdown(f"**Dataset size:** `{rows:,}` rows × `{cols:,}` columns"))


> **Notes**
> - More rows usually means more stable model estimates.
> - A small dataset can make model comparisons noisy.

In [ ]:
# Basic schema overview
df.info()

> **Notes**
> - Categorical columns typically show up as `object`.
> - Numeric columns show up as `int64` or `float64`.


### Inspecting for Missing Values

You should see a **small amount of missing values**.
We injected them on purpose so we can demonstrate missing-value handling.

In [ ]:
# Rich schema summary (types + missingness)
from IPython.display import display
import pandas as pd

schema = pd.DataFrame({
    "column": df.columns,
    "dtype": [str(t) for t in df.dtypes],
    "missing": df.isna().sum().values,
    "missing_%": (df.isna().mean() * 100).round(2).values,
})
schema = schema.sort_values(["missing", "column"], ascending=[False, True]).reset_index(drop=True)

display(
    schema.style
        .format({"missing": "{:,}", "missing_%": "{:.2f}%"})
        .bar(subset=["missing_%"], align="mid")
        .set_caption("Schema summary (sorted by missingness)")
)


> **Notes**
> - Missing values are scattered.
> - This is common in real data pipelines.

## 3) Handling missing data

Missing data is normal in real projects.

Common strategies:
1. **Drop** rows or columns (fast, but you may lose signal)  
2. **Impute** (replace with best guess) missing values (simple and effective for baselines)  
3. **Model-based imputation** (KNN, MICE, etc. — more complex)

In this notebook:
- For **numeric** columns, we will impute using the **mean**.
- For **categorical** columns, we will impute using the **most frequent** category.

In [ ]:
# Missingness percentage (top)
import pandas as pd
from IPython.display import display

missing_pct = (df.isna().mean() * 100).sort_values(ascending=False).reset_index()
missing_pct.columns = ["column", "missing_%"]

display(
    missing_pct.head(12).style
        .format({"missing_%": "{:.2f}%"})
        .bar(subset=["missing_%"], align="mid")
        .set_caption("Missingness percentage (top columns)")
)


> **Notes**
> - If missingness is very high in a column (say 40%+), consider dropping it or collecting better data.
> - Here, missingness is intentionally small. This makes imputation a reasonable choice.

### 3.1 Simple imputation demo (before pipelines)

We will demonstrate imputation on a small subset.
Later, we will do this properly inside a pipeline.

In [ ]:
# SimpleImputer is scikit-learn's standard tool for filling missing values.
from sklearn.impute import SimpleImputer
from IPython.display import display

numeric_cols = ["area", "beds", "bathrooms", "balconies", "area_rate"]
numeric_cols = [c for c in numeric_cols if c in df.columns]

cat_cols = ["city", "furnishing"]
cat_cols = [c for c in cat_cols if c in df.columns]

num_imputer = SimpleImputer(strategy="mean")
cat_imputer = SimpleImputer(strategy="most_frequent")

df_num_demo = pd.DataFrame(num_imputer.fit_transform(df[numeric_cols]), columns=numeric_cols)
df_cat_demo = pd.DataFrame(cat_imputer.fit_transform(df[cat_cols]), columns=cat_cols)

missing_before_after = pd.DataFrame(
    {
        "before_imputation": [int(df[numeric_cols].isna().sum().sum()), int(df[cat_cols].isna().sum().sum())],
        "after_imputation":  [int(df_num_demo.isna().sum().sum()), int(df_cat_demo.isna().sum().sum())],
    },
    index=["numeric (total)", "categorical (total)"]
)
display(missing_before_after)


> **Notes**
> - After imputation, the demo subsets contain **no missing values**.
> - This is exactly what we want before training most models.
> - We will do the same thing again inside a pipeline (the safer approach).
>   
>  **Questions**
> - What is the problem with the approach of missing values?
> - What would you do if the target column has a missing value?

### Demo: Full rows before vs after

In [ ]:

# Choose one numeric and one categorical column to demonstrate
demo_numeric_col = "beds"
demo_categorical_col = "furnishing"

# Get indices where these columns have missing values
num_missing_idx = df[df[demo_numeric_col].isna()].head(5).index
cat_missing_idx = df[df[demo_categorical_col].isna()].head(5).index

# Combine indices (avoid duplicates)
demo_idx = num_missing_idx.union(cat_missing_idx)

# BEFORE imputation (original dataframe)
print("Rows BEFORE imputation")
display(df.loc[demo_idx])

# AFTER imputation (reconstructed dataframe)
df_after_demo = df.copy()
df_after_demo.loc[:, numeric_cols] = df_num_demo
df_after_demo.loc[:, cat_cols] = df_cat_demo

print("Rows AFTER imputation")
display(df_after_demo.loc[demo_idx])


## 4) Summary statistics

We inspect numeric summaries to spot outliers and scale differences.

In [ ]:
# Summary statistics (styled)
import pandas as pd
from IPython.display import display

summary = df.describe(include="all").transpose()
# Show a compact view first (top rows) then let user scroll in output if needed
display(
    summary.style
        .set_caption("Summary statistics (all columns)")
)


> **Note**
> - Rent often has a long right tail (premium listings).
> - Outliers can dominate RMSE.
> - Category counts can be unbalanced (some cities have far more listings).

## 5) Split data into train / validation / test

We use three splits:
- **train** (60%) : fit the model  
- **validation** (20%): compare models / choices  
- **test** (20%) : final unbiased evaluation  

We keep the test set untouched until the end.

In [ ]:
# train_test_split gives a reproducible split.
from sklearn.model_selection import train_test_split
from IPython.display import display

X = df.drop(columns=["rent"])
y = df["rent"]

# 20% test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# From remaining 80%, take 25% as validation -> 20% overall
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, random_state=42
)

split_sizes = pd.DataFrame(
    {
        "rows": [len(X_train), len(X_val), len(X_test)],
        "share": [len(X_train)/len(df), len(X_val)/len(df), len(X_test)/len(df)],
    },
    index=["train", "validation", "test"]
)
split_sizes["share"] = (split_sizes["share"] * 100).round(1).astype(str) + "%"
display(split_sizes)


> **Notes**
> - This is a safe pattern: we do not tune on the test set.
> - Validation is where we compare models.
> - We might want to do different split options

## 6) Exploratory Data Analysis (EDA)

EDA builds intuition.
We ask:
- What does rent look like?
- Which numeric features move rent?
- How do cities and furnishing levels differ?

We will use **Seaborn** for cleaner, more “report-like” visuals.

In [ ]:
# Seaborn sits on top of Matplotlib and gives nicer default plots.
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

sns.set_theme(style="whitegrid")

def format_inr(x, pos=None):
    try:
        return "₹ {:,}".format(int(x))
    except Exception:
        return str(x)

inr_formatter = FuncFormatter(format_inr)

### 6.1 Distribution of rent (histogram + box)

- histogram for shape
- boxplot for outliers

In [ ]:
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(11, 6))
gs = gridspec.GridSpec(2, 1, height_ratios=[4, 1], hspace=0.05)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharex=ax1)

# Matplotlib histogram (avoids seaborn/pandas option incompatibilities)
ax1.hist(y_train, bins=50)
ax1.set_title("Distribution of Monthly Rent (Train Set)")
ax1.set_xlabel("")
ax1.set_ylabel("Count")
ax1.xaxis.set_major_formatter(inr_formatter)

sns.boxplot(x=y_train, ax=ax2)
ax2.set_xlabel("Monthly Rent (₹)")
ax2.xaxis.set_major_formatter(inr_formatter)

plt.show()


> **Inference**
> - The distribution is right-skewed. A few premium rentals exist.
> - The boxplot makes outliers obvious.
> - RMSE will be sensitive to extreme rents.

### 6.2 Numeric features vs rent

We plot rent against each numeric feature.
We also add a trend line to make the direction clearer.

In [ ]:
# %matplotlib widget

import numpy as np

numeric_cols = ["area", "beds", "bathrooms", "balconies", "area_rate"]
numeric_cols = [c for c in numeric_cols if c in df.columns]

# Plot in a grid to reduce scrolling
n = len(numeric_cols)
ncols = 2 if n > 1 else 1
nrows = int(np.ceil(n / ncols))

fig, axes = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(12, 4.8 * nrows),
    constrained_layout=True
)

axes = np.atleast_1d(axes).ravel()

for ax, col in zip(axes, numeric_cols):
    sns.scatterplot(
        data=df,
        x=col,
        y="rent",
        alpha=0.25,
        s=18,
        color="#F58518",
        ax=ax
    )
    ax.yaxis.set_major_formatter(inr_formatter)
    ax.set_title(f"{col} vs Rent")
    ax.set_xlabel(col)
    ax.set_ylabel("Monthly Rent (₹)")

# Hide unused axes
for ax in axes[len(numeric_cols):]:
    ax.set_visible(False)

plt.show()


> **Notes**
> - `area` typically shows the clearest upward trend.
> - Discrete counts (`beds`, `bathrooms`) create “bands”.

### 6.2.1 A 3D view (rent vs area vs area_rate)

A 3D scatter can be a fun way to see how **two** inputs relate to rent at the same time.
Use it for intuition, not for precise conclusions.

In [ ]:
import plotly
import plotly.io as pio

print("Plotly version:", plotly.__version__)
print("Available renderers:", pio.renderers)

# Best defaults for JupyterLab:
# Try one of these (first one usually works)
#pio.renderers.default = "notebook_connected"   # good for JupyterLab + classic
#pio.renderers.default = "jupyterlab"         # works in some setups
pio.renderers.default = "iframe"             # always works, slightly heavier


In [ ]:
import plotly.express as px

# Prepare clean data for 3D visualization
cols3d = ["area", "area_rate", "rent"]
df3d = df[cols3d + ["city"]].dropna().copy()

fig = px.scatter_3d(
    df3d,
    x="area",
    y="area_rate",
    z="rent",
    color="city",                     # remove this line if you want single-color points
    opacity=0.5,
    title="3D View: Area vs Area Rate vs Rent",
)

fig.update_layout(
    scene=dict(
        xaxis_title="Area (sqft)",
        yaxis_title="Area Rate",
        zaxis_title="Monthly Rent (₹)",
    ),
    margin=dict(l=0, r=0, b=0, t=40),
)

fig.show()


> **Notes**
> - Listings with similar area can have very different rent if the area rate differs.
> - This supports why both `area` and `area_rate` can be useful features.

### 6.3 Categorical overview (counts)

We check how many categories exist.
Then we visualize the top counts.

In [ ]:
import pandas as pd
from IPython.display import display

# Explicitly include both legacy object and new string dtypes
categorical_cols = df.select_dtypes(include=["object", "string"]).columns.tolist()

cardinality = (
    pd.DataFrame({
        "column": categorical_cols,
        "unique_categories": [df[c].nunique(dropna=True) for c in categorical_cols]
    })
    .sort_values("unique_categories", ascending=False)
)

display(
    cardinality.style
        .format({"unique_categories": "{:,}"})
        .bar(subset=["unique_categories"], align="mid")
        .set_caption("Category counts (unique values)")
)


> **Inference**
> - `city` and `furnishing` are low-cardinality. They are great for one-hot encoding.
> - `locality` can have many categories. We will skip it in the first baseline model.

In [ ]:
top_cities = (
    df["city"]
    .value_counts()
    .head(12)
    .reset_index()
)
top_cities.columns = ["city", "count"]

# Plot in a grid to reduce scrolling
fig, axes = plt.subplots(1, 2, figsize=(14, 5.5), constrained_layout=True)

sns.barplot(
    data=top_cities,
    x="city",
    y="count",
    ax=axes[0]
)
axes[0].set_title("Top Cities by Listing Count")
axes[0].set_xlabel("City")
axes[0].set_ylabel("Number of Listings")
axes[0].tick_params(axis="x", rotation=25)

sns.countplot(
    data=df,
    x="furnishing",
    order=df["furnishing"].value_counts().index,
    ax=axes[1]
)
axes[1].set_title("Furnishing Distribution")
axes[1].set_xlabel("Furnishing")
axes[1].set_ylabel("Count")
axes[1].tick_params(axis="x", rotation=15)

plt.show()


> **Notes**
> - If one city dominates, the model may become city-biased.
> - Later, check error by city.
> - Furnishing often shifts the rent baseline. This is a strong reason to keep it as a feature.

### 6.4 Rent by category (boxplots)

Boxplots show how rent spreads across categories.
We plot the top cities for readability.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

categorical_cols = ["city", "furnishing"]
categorical_cols = [c for c in categorical_cols if c in df.columns]

fig, axes = plt.subplots(1, len(categorical_cols), figsize=(14, 5), constrained_layout=True)

# Handle single-plot case
if len(categorical_cols) == 1:
    axes = [axes]

for ax, col in zip(axes, categorical_cols):
    sns.boxplot(
        data=df,
        x=col,
        y="rent",
        hue=col,
        palette="pastel",
        showfliers=False,
        legend=False,
        ax=ax
    )
    ax.yaxis.set_major_formatter(inr_formatter)
    ax.set_title(f"Rent Distribution by {col}")
    ax.set_xlabel(col)
    ax.set_ylabel("Monthly Rent (₹)")
    ax.tick_params(axis="x", rotation=25)

plt.show()


> **Note**
> - Some cities have a clearly higher rent range.
> - A global model needs to learn both city-level shifts and within-city trends.

## Key insights from EDA

Before we model, we write down a few observations from the charts.
This keeps the workflow realistic: **EDA should influence decisions**.

> **Notes**
> - Note the top cities. A model may learn dominant cities better than rare ones.
> - Compare median rent across furnishing levels. If the gaps are large, furnishing should help prediction.
> - Check which numeric feature correlates most with rent (often `area` or `area_rate`).

## 7) Correlation: Understanding Relationships Between Features

Correlation is a quick way to rank numeric relationships. 

**Correlation** measures how strongly two numeric variables move **together**.

* A **positive correlation** means both variables increase together
* A **negative correlation** means one increases while the other decreases
* A correlation close to **0** means little or no linear relationship

Correlation values typically range from **-1 to +1**.

### How Correlation Is Computed (Intuition)

For numeric features, correlation compares:

* how much each value deviates from its mean
* whether those deviations move in the same or opposite direction

In practice, we commonly use **Pearson correlation**, which captures **linear relationships** between variables.

### Why We Look at Correlation

Correlation helps us:

* understand which features are related to the target
* detect redundant or highly related features
* get an early sense of which variables may be useful for modeling

> Correlation does **not** imply causation — it’s an exploratory signal, not a conclusion.

### What We’ll Do Next

We will compute **correlation among numeric features**, including the target (`rent`), and visualize it to better understand their relationships.


In [ ]:
# Correlation with rent (numeric only)
import pandas as pd
from IPython.display import display

corr = df[numeric_cols + ["rent"]].corr(numeric_only=True)

corr_with_rent = corr["rent"].drop("rent").sort_values(ascending=False).reset_index()
corr_with_rent.columns = ["feature", "correlation_with_rent"]

display(
    corr_with_rent.style
        .format({"correlation_with_rent": "{:.3f}"})
        .bar(subset=["correlation_with_rent"], align="mid")
        .set_caption("Numeric correlation with rent (higher magnitude → stronger linear relationship)")
)


### 7.1 Correlation heatmap (numeric features)

A heatmap makes correlation patterns easier to scan.
We only include numeric columns here.

In [ ]:
# A heatmap is a compact way to view correlations.
# We keep it to numeric columns to avoid mixing types.
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True, cbar=True)
plt.title("Correlation Heatmap (Numeric Features)")
plt.show()

## 8) Feature selection (baseline model)

We start with a small, stable set of features.

We use:
- numeric: `area`, `beds`, `bathrooms`, `balconies`, `area_rate`
- categorical: `city`, `furnishing`

We skip for now:
- `locality` (high-cardinality)
- `house_type` (free text)

Future idea:
- Use `locality` to map to a **pincode** or zone, then use grouped features.

In [ ]:
num_features = ["area", "beds", "bathrooms", "balconies", "area_rate"]
cat_features = ["city", "furnishing"]

X_train_sel = X_train[num_features + cat_features].copy()
X_val_sel   = X_val[num_features + cat_features].copy()
X_test_sel  = X_test[num_features + cat_features].copy()

X_train_sel.head()

> **Notes**
> - Baselines should be simple and explainable.
> - You can add complexity later once you trust the pipeline.

## 9) Preprocessing with pipelines (imputation + one-hot)

A pipeline chains steps safely.
It also avoids data leakage.

Our preprocessor will:
- impute numeric missing values with the **mean**
- impute categorical missing values with the **most frequent**
- one-hot encode categorical columns

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
])

categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(transformers=[
    ("num", numeric_pipeline, num_features),
    ("cat", categorical_pipeline, cat_features),
])

preprocess

### 9.1 One-hot encoding (how the data looks)

One-hot encoding turns each category into its own binary (0/1) column.  
This lets linear and tree-based models work with categorical inputs.

In [ ]:
from IPython.display import display

# Show a tiny sample BEFORE encoding (raw categorical columns present)
sample_before = X_train_sel.head(3).copy()
display(sample_before)

# Fit the preprocessing on training data and show a small AFTER view
X_after = preprocess.fit_transform(X_train_sel)

# Build readable feature names (numeric + one-hot category names)
ohe = preprocess.named_transformers_["cat"].named_steps["onehot"]
ohe_feature_names = ohe.get_feature_names_out(cat_features)

feature_names = list(num_features) + list(ohe_feature_names)

# Convert a small slice to a DataFrame for display
sample_after = pd.DataFrame(X_after[:3], columns=feature_names)
display(sample_after)


> **Notes**
> - We do not clean the DataFrame by hand.
> - The pipeline learns imputation values from training data only.
> - `handle_unknown="ignore"` makes inference safer if a new city appears later.

## 10) Cross-validation (CV) for model comparison

Cross-validation trains the model multiple times on different folds of the training set.
This reduces the risk of a lucky or unlucky split.

We compare three models:

| Model                       | What is it?                                           | Key Idea                                                  | What it Optimizes                              | When It Works Well                                                 |
| --------------------------- | ----------------------------------------------------- | --------------------------------------------------------- | ---------------------------------------------- | ------------------------------------------------------------------ |
| **Linear Regression**       | Models the target as a weighted sum of input features | Assumes a linear relationship between features and target | Minimizes Mean Squared Error (MSE)             | When relationships are roughly linear and features are informative |
| **Ridge Regression**        | Linear Regression with coefficient regularization     | Penalizes large coefficients to reduce overfitting        | MSE + L2 penalty on coefficients               | When features are correlated or feature space is high-dimensional  |
| **Decision Tree Regressor** | Learns a set of if–else rules to make predictions     | Recursively splits data to reduce prediction error        | Reduces variance (squared error) at each split | When relationships are non-linear or feature interactions matter   |



### 📏 Regression Evaluation Metrics

| Metric                        | What It Measures                                                  | Interpretation                                  |
| ----------------------------- | ----------------------------------------------------------------- | ----------------------------------------------- |
| **MAE** (Mean Absolute Error) | Average absolute difference between predictions and actual values | Easy to interpret, same units as the target (₹) |
| **MSE** (Mean Squared Error)  | Average of squared prediction errors                              | Penalizes large errors more heavily             |
| **R²** (R-squared)            | How much of the variation in the target is explained by the model | Closer to 1 means better explanatory power      |


### Quick Notes

* **MAE** → “On average, how wrong are we?”
* **MSE** → “How bad are large mistakes?”
* **R²** → “How well does the model explain the data?”

> Always look at **error metrics (MAE/MSE)** along with **R²**.

If you want an even tighter 2-row version or a spoken-script version, I can do that too.


In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
import numpy as np

### Cross-Validation (Why We Use It)

Cross-validation helps us **estimate how well a model will perform on unseen data**.

Instead of training and evaluating the model on a single split:

* the data is split into multiple folds
* the model is trained and evaluated multiple times
* results are averaged for a more reliable estimate

This reduces the risk of:

* depending too much on one lucky (or unlucky) split
* overestimating model performance

> Cross-validation gives a **more stable and trustworthy evaluation**.


In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

candidates = {
    "LinearRegression": LinearRegression(),
    "Ridge(alpha=1.0)": Ridge(alpha=1.0),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
}

def cv_mae_scores(model, X, y):
    from sklearn.pipeline import Pipeline
    pipe = Pipeline(steps=[("preprocess", preprocess), ("model", model)])
    scores = cross_val_score(pipe, X, y, cv=cv, scoring="neg_mean_absolute_error")
    return -scores  # convert to positive MAE

cv_scores = {name: cv_mae_scores(model, X_train_sel, y_train) for name, model in candidates.items()}

cv_scores_df = pd.DataFrame(cv_scores)
summary = pd.DataFrame(
    {
        "MAE (mean)": cv_scores_df.mean().astype(float),
        "MAE (std)": cv_scores_df.std(ddof=0).astype(float),
    }
).sort_values("MAE (mean)")

display(cv_scores_df.style.format(precision=2))
display(summary.style.format(precision=3))


> **Notes**
> - You see fold-by-fold RMSE for each model.
> - Lower MAE is better.
> - High spread across folds means instability.

In [ ]:
# Visualize CV score distributions (long-form)
cv_long = cv_scores_df.melt(var_name="model", value_name="mae")

plt.figure(figsize=(11, 6))
ax = sns.boxplot(data=cv_long, x="model", y="mae", hue="model", palette="Set3")
if ax.get_legend() is not None:
    ax.get_legend().remove()

plt.title("Cross-Validation MAE by Model (Train Set)")
plt.ylabel("MAE")
plt.xlabel("")
plt.xticks(rotation=10)
plt.show()


> **Notes**
> - Prefer models with low mean RMSE and low variance.
> - Trees can overfit and show higher variance.

## 11) Train each model and evaluate on the validation set

Now we train on the training split once.
Then we score on the validation split.
This is a realistic model-selection loop.

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

def train_and_eval(model, X_tr, y_tr, X_eval, y_eval):
    from sklearn.pipeline import Pipeline
    pipe = Pipeline(steps=[("preprocess", preprocess), ("model", model)])
    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_eval)
    mae = float(mean_absolute_error(y_eval, preds))
    r2 = float(r2_score(y_eval, preds))
    return pipe, preds, mae, r2

val_results = []
val_preds = {}
trained = {}

for name, model in candidates.items():
    pipe, preds, mae, r2 = train_and_eval(model, X_train_sel, y_train, X_val_sel, y_val)
    trained[name] = pipe
    val_preds[name] = preds
    val_results.append((name, mae, r2))

val_table = pd.DataFrame(val_results, columns=["model", "val_MAE", "val_R2"]).sort_values("val_MAE")
display(val_table.style.format({"val_MAE": "{:.2f}", "val_R2": "{:.3f}"}))


> **Notes**
> - Validation MAE is easiest to interpret in ₹.
> - R² is a sanity check. Do not optimize for it blindly.

### 11.1 Validation residuals (per model)

Residual = actual − predicted.
We want residuals centered around 0 with reasonable spread.

In [ ]:
plt.figure(figsize=(12, 6))
for name, preds in val_preds.items():
    if (name == 'LinearRegression') or (name == 'DecisionTree') or (name == 'Ridge(alpha=1.0)'):
        residuals = y_val.values - preds
        plt.hist(residuals, bins=35, histtype="step", linewidth=2, label=name)

plt.gca().xaxis.set_major_formatter(inr_formatter)
plt.title("Validation Residual Distributions (Actual − Predicted)")
plt.xlabel("Residual (₹)")
plt.ylabel("Count")
plt.legend()
plt.show()


> **Note**
> - Narrower residual distributions usually mean lower error.
> - Long tails usually come from premium listings and outliers.

## 12) Pick the best model and evaluate on the test set

We pick the model with the best validation RMSE.
Then we refit it on train + validation.
Finally, we evaluate once on the test set.

In [ ]:
best_name, best_mae, best_r2 = val_table.iloc[0]["model"], float(val_table.iloc[0]["val_MAE"]), float(val_table.iloc[0]["val_R2"])

best_summary = pd.DataFrame(
    [{"best_model": best_name, "best_val_MAE": best_mae, "best_val_R2": best_r2}]
)
display(best_summary.style.format({"best_val_MAE": "{:.2f}", "best_val_R2": "{:.3f}"}))


> **Notes**
> - After this point, we stop comparing models.
> - The test set is used only once.

In [ ]:
best_model = candidates[best_name]

X_trainval_sel = X_trainval[num_features + cat_features].copy()

from sklearn.pipeline import Pipeline
final_pipe = Pipeline(steps=[("preprocess", preprocess), ("model", best_model)])
final_pipe.fit(X_trainval_sel, y_trainval)

test_preds = final_pipe.predict(X_test_sel)

test_mae = float(mean_absolute_error(y_test, test_preds))
test_r2 = float(r2_score(y_test, test_preds))

test_summary = pd.DataFrame(
    [{"model": best_name, "test_MAE": test_mae, "test_R2": test_r2}]
)
display(test_summary.style.format({"test_MAE": "{:.2f}", "test_R2": "{:.3f}"}))


> **Note**
> - This test RMSE is your final unbiased estimate.
> - Do not tune hyperparameters using the test set.

## 13) Error analysis on the test set

We inspect:
- residual distribution
- biggest mistakes (largest absolute error)
- average error by city (quick check)

In [ ]:
test_residuals = y_test.values - test_preds

plt.figure(figsize=(11, 5.5))
plt.hist(test_residuals, bins=40)
plt.gca().xaxis.set_major_formatter(inr_formatter)
plt.title("Test Residual Distribution (Actual − Predicted)")
plt.xlabel("Residual (₹)")
plt.ylabel("Count")
plt.show()


> **Notes**
> - If residuals are shifted left or right, the model is biased.
> - Wide spread means the model is not precise.
> - Long tails indicate outliers.

In [ ]:
# Biggest errors (top 15)
from IPython.display import display
import pandas as pd
import numpy as np

# Reconstruct the error table explicitly (safe if cells are run out of order)
errors = X_test_sel.copy()
errors["actual_rent"] = y_test.values
errors["predicted_rent"] = test_preds
errors["abs_error"] = np.abs(errors["actual_rent"] - errors["predicted_rent"])

top_err = errors.sort_values("abs_error", ascending=False).head(15).copy()

display(
    top_err.style
        .format({
            "actual_rent": "₹ {:,.0f}",
            "predicted_rent": "₹ {:,.0f}",
            "abs_error": "₹ {:,.0f}",
        })
        .bar(subset=["abs_error"], align="mid")
        .set_caption("Largest absolute errors on the test set")
)


> **Notes**
> - This table is actionable.
> - It shows which listings need better features or a stronger model.

In [ ]:
if "city" in errors.columns:
    city_mae = (
        errors.groupby("city")["abs_error"]
        .mean()
        .sort_values(ascending=False)
    )

    plt.figure(figsize=(10.5, 5.5))
    sns.barplot(
        x=city_mae.index[:10],
        y=city_mae.values[:10],
        hue=city_mae.index[:10],   # explicitly map color
        palette="magma",
        legend=False               # avoid redundant legend
    )
    plt.gca().yaxis.set_major_formatter(inr_formatter)
    plt.title("Average Absolute Error by City — Test Set")
    plt.xlabel("City")
    plt.ylabel("Avg Absolute Error (₹)")
    plt.xticks(rotation=25, ha="right")
    plt.show()

    city_mae.head(10)

# Also show the table for quick reading
from IPython.display import display
import pandas as pd

city_mae_table = city_mae.head(10).reset_index()
city_mae_table.columns = ["city", "avg_abs_error"]

display(
    city_mae_table.style
        .format({"avg_abs_error": "₹ {:,.0f}"})
        .bar(subset=["avg_abs_error"], align="mid")
        .set_caption("Avg absolute error by city (top 10) — test set")
)


> **Note**
> - If one city has much higher error, you may need more data or better features for that city.
> - This is a common production issue for geo-based models.

## 14) Recap and next steps

We built a realistic ML workflow:
- EDA with readable plots
- explicit missing data handling section
- train / validation / test split
- preprocessing pipeline (impute + one-hot)
- model comparison using cross-validation
- final evaluation on the test set
- error analysis

**Next steps**
- Try stronger models (RandomForest, GradientBoosting)
- Engineer features (log(rent), rent per sqft)
- Add locality carefully (high-cardinality strategies)
- Map locality to zones/pincodes for richer location signals

## Saving the Best Model (Deployment Prep)

Training a model is only half the job — **deployment is where most real work happens**.
Typically you must ensure:
- the **same preprocessing** runs in production
- requests are handled with **reliability and low latency**
- the system supports **monitoring, drift detection, and retraining**
- the service is **versioned, tested, and scalable** (MLOps)

Next, we’ll save the **full pipeline (preprocess + model)** so it can be loaded directly in a FastAPI service.


In [ ]:
from sklearn.pipeline import Pipeline
import joblib
from pathlib import Path
from datetime import datetime

# 1) Choose the best model (set this based on your selection)
# Example:
# best_model = Ridge(alpha=1.0)
# best_model = DecisionTreeRegressor(max_depth=6, random_state=42)

# 2) Build the full pipeline (preprocess + model)
best_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", best_model),
])

# 3) Fit on training data (or train+val if you decide later)
best_pipeline.fit(X_train_sel, y_train)

# 4) Save to disk
artifacts_dir = Path("artifacts")
artifacts_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_path = artifacts_dir / f"rent_model_pipeline_{timestamp}.joblib"

joblib.dump(best_pipeline, model_path)

print(f"✅ Saved model pipeline to: {model_path}")
print("Includes preprocessing + model (ready to load in FastAPI).")


### Quick “load and predict”

In [ ]:
loaded = joblib.load(model_path)
sample_preds = loaded.predict(X_train_sel.head(3))
print(sample_preds)


## 🧠 Optional: Interpreting the Decision Tree Model

At this point, we have a trained pipeline that includes both:
- preprocessing (imputation + one-hot encoding)
- the final Decision Tree model

In this optional section, we inspect **feature importance** to understand:
- which inputs have the biggest impact on predictions
- why changing some inputs (e.g., `city`) may not change the predicted rent

Note: With one-hot encoding, a single categorical feature (like `city`) becomes many binary columns.
So we will look at importance in two ways:
1. top transformed features (after preprocessing)
2. importance aggregated back to the original feature names


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# --- Extract fitted preprocessing + trained tree from your pipeline ---
preprocess_fitted = best_pipeline.named_steps["preprocess"]
tree = best_pipeline.named_steps["model"]

# --- Get feature names after preprocessing and importance scores ---
feature_names = preprocess_fitted.get_feature_names_out()
importances = tree.feature_importances_

feature_importance_df = (
    pd.DataFrame({"feature": feature_names, "importance": importances})
    .sort_values("importance", ascending=False)
)

print("Top 15 transformed features used by the tree:")
display(feature_importance_df.head(15))

# --- Correctly map transformed feature names back to original features ---
# Remove "num__" / "cat__" prefix
clean = feature_importance_df["feature"].str.replace(r"^(num__|cat__)", "", regex=True)

def base_feature_name(transformed_name: str, cat_cols) -> str:
    """
    Map names like:
      - city_Bangalore -> city
      - furnishing_Fully Furnished -> furnishing
      - area_rate -> area_rate  (do NOT split numeric names with underscores)
    """
    for c in cat_cols:
        if transformed_name == c or transformed_name.startswith(c + "_"):
            return c
    return transformed_name

# cat_features should already exist in your notebook; if not, define it consistently
# cat_features = ["city", "furnishing"]  # example
feature_importance_df["base_feature"] = clean.apply(lambda s: base_feature_name(s, cat_features))

# --- Aggregate importance back to original feature level ---
grouped_importance = (
    feature_importance_df.groupby("base_feature")["importance"]
    .sum()
    .sort_values(ascending=False)
)

print("Total importance by original feature:")
display(grouped_importance.to_frame("total_importance"))

# --- Plot top 10 original features ---
top_k = 10
grouped_importance.head(top_k).sort_values().plot(kind="barh", figsize=(8, 4))
plt.title("Decision Tree Feature Importance (Aggregated)")
plt.xlabel("Total Importance")
plt.show()


### 🎯 Key Takeaway

> Feature importance tells us what the **model learned**,
> not what we **assumed** would be important.

This step helps us:

* trust the model
* explain predictions
* identify missing or redundant features
